### Imports


In [36]:
import time
import torch
import numpy as np
from typing import List
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from TorchCRF import CRF

%run preprocessing.ipynb

474
['ق', 'و', 'ل', 'ه', 'ل', 'ع', 'د', 'م', 'م', 'ا', 'ت', 'ت', 'ع', 'ل', 'ق', 'إ', 'ل', 'خ', 'أ', 'ي', 'ا', 'ل', 'و', 'ص', 'ي', 'ة', 'ق', 'و', 'ل', 'ه', 'م', 'ا', 'م', 'ر', 'أ', 'ي', 'ق', 'ب', 'ي', 'ل', 'ق', 'و', 'ل', 'ا', 'ل', 'م', 'ت', 'ن', 'ل', 'غ', 'ت', 'و', 'ل', 'و', 'ا', 'ق', 'ت', 'ص', 'ر', 'ع', 'ل', 'ى', 'أ', 'و', 'ص', 'ي', 'ت', 'ل', 'ه', 'ب', 'ش', 'ا', 'ة', 'أ', 'و', 'أ', 'ع', 'ط', 'و', 'ه', 'ش', 'ا', 'ة', 'و', 'ل', 'ا', 'غ', 'ن', 'م', 'ل', 'ه', 'ع', 'ن', 'د', 'ا', 'ل', 'م', 'و', 'ت', 'ه', 'ل', 'ت', 'ب', 'ط', 'ل', 'ا', 'ل', 'و', 'ص', 'ي', 'ة', 'أ', 'و', 'ي', 'ش', 'ت', 'ر', 'ى', 'ل', 'ه', 'ش', 'ا', 'ة', 'و', 'ي', 'ؤ', 'خ', 'ذ', 'م', 'ن', 'ق', 'و', 'ل', 'ه', 'ا', 'ل', 'آ', 'ت', 'ي', 'ك', 'م', 'ا', 'ل', 'و', 'ل', 'م', 'ي', 'ق', 'ل', 'م', 'ن', 'م', 'ا', 'ل', 'ي', 'و', 'ل', 'ا', 'م', 'ن', 'غ', 'ن', 'م', 'ي', 'أ', 'ن', 'ه', 'ا', 'ل', 'ا', 'ت', 'ب', 'ط', 'ل', 'و', 'ع', 'ب', 'ا', 'ر', 'ة', 'ا', 'ل', 'ك', 'ن', 'ز', 'و', 'ل', 'و', 'ل', 'م', 'ي', 'ق', 'ل', 'م', 'ن', 'م', 'ا', 'ل', 'ي', 

### Constants


In [37]:
EMBEDDING_DIM = 200
HIDDEN_SIZE = 512
NUM_LAYERS = 1
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 256
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)

TRAIN_PATH = "../dataset/train.txt"
VAL_PATH = "../dataset/val.txt"
RNN_PATH = "./models/rnn.pth"
CNN_PATH = "./models/cnn.pth"
CRF_PATH = "./models/crf.pth"
CBHG_PATH = "./models/cbhg.pth"

### Intialize the device


In [38]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))

else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

No GPU available, using the CPU instead.


### Model building


### RNN


In [39]:
class RNN(nn.Module):
    def __init__(
        self,
        vocab_size=VOCAB_SIZE,
        n_classes=LABELS_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
    ):
        """
        The constructor of our RNN model
        Inputs:
        - vacab_size: the number of unique characters
        - embedding_dim: the embedding dimension
        - n_classes: the number of final classes (diacritics)
        """
        super(RNN, self).__init__()

        # (1) Create an embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )

        # (3) Create a linear layer with number of neorons = n_classes
        self.linear = nn.Linear(hidden_size * 2, n_classes)

    def forward(self, sentences):
        """
        This function does the forward pass of our model
        Inputs:
        - sentences: tensor of shape (batch_size, max_length)

        Returns:
        - final_output: tensor of shape (batch_size, max_length, n_classes)
        """

        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)

        return output

### CNN


In [40]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size=VOCAB_SIZE,
        num_classes=LABELS_SIZE,
        embedding_dim=EMBEDDING_DIM,
        filter_sizes=[3, 4, 5],
        num_filters=[100, 100, 100],
        dropout=0.5,
    ):
        super(CNN, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Conv Network
        self.conv1d_list = nn.ModuleList(
            [
                nn.Conv1d(
                    in_channels=embedding_dim,
                    out_channels=num_filters[i],
                    kernel_size=filter_sizes[i],
                )
                for i in range(len(filter_sizes))
            ]
        )

        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [
            F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in x_conv_list
        ]

        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list], dim=1)

        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

### CRF


In [41]:
class LSTM_CRF(nn.Module):
    def __init__(
        self,
        vocab_size=VOCAB_SIZE,
        n_classes=LABELS_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
    ):
        super(LSTM_CRF, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )

        # Linear layer
        self.linear = nn.Linear(hidden_size * 2, n_classes)

        # CRF layer
        self.crf = CRF(n_classes)  # Place the CRF layer after the linear layer

    def forward(self, sentences):
        embeddings = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeddings)
        output = self.linear(lstm_out)
        return output  # Return raw output for CRF loss calculation

    def predict(self, sentences):
        output = self.forward(sentences)
        predictions = self.crf.decode(output)
        return predictions

### CBHG


In [ ]:
class Prenet(nn.Module):
    """
    A prenet is a collection of linear layers with dropout(0.5),
    and RELU activation function
    Args:
    config: the hyperparameters object
    in_dim (int): the input dim
    """

    def __init__(
        self, in_dim: int, prenet_depth: List[int] = [256, 128], dropout: int = 0.5
    ):
        """Initializing the prenet module"""
        super().__init__()
        in_sizes = [in_dim] + prenet_depth[:-1]
        self.layers = nn.ModuleList(
            [
                nn.Linear(in_size, out_size)
                for (in_size, out_size) in zip(in_sizes, prenet_depth)
            ]
        )
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs: torch.Tensor):
        """Calculate forward propagation
        Args:
        inputs (batch_size, seqLen): the inputs to the prenet, the input shapes could
        be different as it is being used in both encoder and decoder.
        Returns:
        Tensor: the output of  the forward propagation
        """
        for linear in self.layers:
            inputs = self.dropout(self.relu(linear(inputs)))
        return inputs


class Highway(nn.Module):
    """Highway Networks were developed by (Srivastava et al., 2015)
    to overcome the difficulty of training deep neural networks
    (https://arxiv.org/abs/1507.06228).
    Args:
    in_size (int): the input size
    out_size (int): the output size
    """

    def __init__(self, in_size, out_size):
        """
        Initializing Highway networks
        """
        super().__init__()
        self.H = nn.Linear(in_size, out_size)
        self.H.bias.data.zero_()
        self.T = nn.Linear(in_size, out_size)
        self.T.bias.data.fill_(-1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs: torch.Tensor):
        """Calculate forward propagation
        Args:
        inputs (Tensor):
        """
        H = self.relu(self.H(inputs))
        T = self.sigmoid(self.T(inputs))
        return H * T + inputs * (1.0 - T)


class BatchNormConv1d(nn.Module):
    """
    A nn.Conv1d followed by an optional activation function, and nn.BatchNorm1d
    """

    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        kernel_size: int,
        stride: int,
        padding: int,
        activation=None,
    ):
        super().__init__()
        self.conv1d = nn.Conv1d(
            in_dim,
            out_dim,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_dim)
        self.activation = activation

    def forward(self, x):
        x = self.conv1d(x)
        if self.activation is not None:
            x = self.activation(x)
        return self.bn(x)


class CBHG(nn.Module):
    """The CBHG module (1-D Convolution Bank + Highway network + Bidirectional GRU)
    was proposed by (Lee et al., 2017, https://www.aclweb.org/anthology/Q17-1026)
    for a character-level NMT model.
    It was adapted by (Wang et al., 2017) for building the Tacotron.
    It is used in both the encoder and decoder  with different parameters.
    """

    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        K: int,
        projections,
        highway_layers: int = 4,
    ):
        """Initializing the CBHG module
        Args:
        in_dim (int): the input size
        out_dim (int): the output size
        k (int): number of filters
        projections: A list of integers representing the output sizes of convolutional projections.
        highway_layers: Number of highway layers.
        """
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim
        self.relu = nn.ReLU()
        # list of modules each one of them will be BatchNoemConv1
        # and each of them has a kernel size ranging from 1 -> k
        self.conv1d_banks = nn.ModuleList(
            [
                BatchNormConv1d(
                    in_dim,
                    in_dim,
                    kernel_size=k,
                    stride=1,
                    padding=k // 2,
                    activation=self.relu,
                )
                for k in range(1, K + 1)
            ]
        )
        """
        kernel_size: The size of the window over which the maximum value is computed. It specifies the size of the pooling operation.
        stride: The step size to slide the pooling window along the input. If not specified, it defaults to kernel_size.
        padding: Zero-padding added to both sides of the input. Padding helps to control the spatial dimensions of the output.
        """
        self.max_pool1d = nn.MaxPool1d(kernel_size=2, stride=1, padding=1)

        in_sizes = [K * in_dim] + projections[:-1]
        activations = [self.relu] * (len(projections) - 1) + [None]
        self.conv1d_projections = nn.ModuleList(
            [
                BatchNormConv1d(
                    in_size, out_size, kernel_size=3, stride=1, padding=1, activation=ac
                )
                for (in_size, out_size, ac) in zip(in_sizes, projections, activations)
            ]
        )

        self.pre_highway = nn.Linear(projections[-1], in_dim, bias=False)
        self.highways = nn.ModuleList([Highway(in_dim, in_dim) for _ in range(4)])

        self.gru = nn.GRU(in_dim, out_dim, 1, batch_first=True, bidirectional=True)

    def forward(self, inputs, input_lengths=None):
        # (B, T_in, in_dim)
        x = inputs
        x = x.transpose(1, 2)
        T = x.size(-1)

        # (B, in_dim*K, T_in)
        # Concat conv1d bank outputs
        x = torch.cat([conv1d(x)[:, :, :T] for conv1d in self.conv1d_banks], dim=1)
        assert x.size(1) == self.in_dim * len(self.conv1d_banks)
        x = self.max_pool1d(x)[:, :, :T]

        for conv1d in self.conv1d_projections:
            x = conv1d(x)

        # (B, T_in, in_dim)
        # Back to the original shape
        x = x.transpose(1, 2)

        if x.size(-1) != self.in_dim:
            x = self.pre_highway(x)

        # Residual connection
        x += inputs
        for highway in self.highways:
            x = highway(x)

        if input_lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, input_lengths, batch_first=True)

        # (B, T_in, in_dim*2)
        self.gru.flatten_parameters()
        outputs, _ = self.gru(x)

        if input_lengths is not None:
            outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        return outputs


class CBHGModel(nn.Module):
    """CBHG model implementation as described in the paper:
     https://ieeexplore.ieee.org/document/9274427

    Args:
    inp_vocab_size (int): the number of the input symbols
    targ_vocab_size (int): the number of the target symbols (diacritics)
    embedding_dim (int): the embedding  size
    use_prenet (bool): whether to use prenet or not
    prenet_sizes (List[int]): the sizes of the prenet networks
    cbhg_gru_units (int): the number of units of the CBHG GRU, which is the last
    layer of the CBHG Model.
    cbhg_filters (int): number of filters used in the CBHG module
    cbhg_projections: projections used in the CBHG module

    Returns:
    diacritics Dict[str, Tensor]:
    """

    def __init__(
        self,
        inp_vocab_size: int,
        targ_vocab_size: int,
        embedding_dim: int = 512,
        use_prenet: bool = True,
        prenet_sizes=[512, 256],
        cbhg_gru_units: int = 512,
        cbhg_filters: int = 16,
        cbhg_projections=[128, 256],
        post_cbhg_layers_units=[256, 256],
        post_cbhg_use_batch_norm: bool = True,
    ):
        super().__init__()
        self.use_prenet = use_prenet
        self.embedding = nn.Embedding(inp_vocab_size, embedding_dim)
        if self.use_prenet:
            self.prenet = Prenet(embedding_dim, prenet_depth=prenet_sizes)

        self.cbhg = CBHG(
            prenet_sizes[-1] if self.use_prenet else embedding_dim,
            cbhg_gru_units,
            K=cbhg_filters,
            projections=cbhg_projections,
        )

        layers = []
        post_cbhg_layers_units = [cbhg_gru_units] + post_cbhg_layers_units

        for i in range(1, len(post_cbhg_layers_units)):
            layers.append(
                nn.LSTM(
                    post_cbhg_layers_units[i - 1] * 2,
                    post_cbhg_layers_units[i],
                    bidirectional=True,
                    batch_first=True,
                )
            )
            if post_cbhg_use_batch_norm:
                layers.append(nn.BatchNorm1d(post_cbhg_layers_units[i] * 2))

        self.post_cbhg_layers = nn.ModuleList(layers)
        self.projections = nn.Linear(post_cbhg_layers_units[-1] * 2, targ_vocab_size)
        self.post_cbhg_layers_units = post_cbhg_layers_units
        self.post_cbhg_use_batch_norm = post_cbhg_use_batch_norm

    def forward(
        self,
        src: torch.Tensor,
        lengths=None,
        target=None,  # not required in this model
    ):
        """Compute forward propagation"""

        # src = [batch_size, src len]
        # lengths = [batch_size]
        # target = [batch_size, trg len]

        embedding_out = self.embedding(src)
        # embedding_out; [batch_size, src_len, embedding_dim]

        cbhg_input = embedding_out
        if self.use_prenet:
            cbhg_input = self.prenet(embedding_out)

            # cbhg_input = [batch_size, src_len, prenet_sizes[-1]]

        outputs = self.cbhg(cbhg_input, lengths)

        hn = torch.zeros((2, 2, 2))
        cn = torch.zeros((2, 2, 2))

        for i, layer in enumerate(self.post_cbhg_layers):
            if isinstance(layer, nn.BatchNorm1d):
                outputs = layer(outputs.permute(0, 2, 1))
                outputs = outputs.permute(0, 2, 1)
                continue
            if i > 0:
                outputs, (hn, cn) = layer(outputs, (hn, cn))
            else:
                outputs, (hn, cn) = layer(outputs)

        predictions = self.projections(outputs)

        # predictions = [batch_size, src len, targ_vocab_size]

        # output = {"diacritics": predictions}

        return predictions

### Train


In [42]:
def data_loader(
    train_inputs, val_inputs, train_labels, val_labels, batch_size=BATCH_SIZE
):
    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data, sampler=train_sampler, batch_size=batch_size
    )

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader


# Specify loss function
loss_fn = nn.CrossEntropyLoss()


def train(
    path, model, optimizer, train_dataloader, val_dataloader=None, epochs=NUM_EPOCHS
):
    """Train the model"""

    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(
        f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}"
    )
    print("-" * 60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass
            output = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(output.view(-1, output.shape[-1]), b_labels.view(-1))
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                torch.save(model.state_dict(), path)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(
                f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}"
            )

    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")


def evaluate(model, val_dataloader):
    """
    After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Get the output
        with torch.no_grad():
            output = model(b_input_ids)

        # Compute loss
        loss = loss_fn(output.view(-1, output.shape[-1]), b_labels.view(-1))
        val_loss.append(loss.item())

        # Get the predictions
        preds = output.argmax(dim=2)

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

### Intialize the models


In [43]:
def initilize_model(model_type, learning_rate=LEARNING_RATE):
    # Instantiate the model
    if model_type == "CNN":
        model = CNN()
        path = CNN_PATH
    elif model_type == "RNN":
        model = RNN()
        path = RNN_PATH
    elif model_type == "CRF":
        model = LSTM_CRF()
        path = CRF_PATH
    elif model_type == "CBHG":
        model = CBHGModel(
            embedding_dim=EMBEDDING_DIM,
            inp_vocab_size=VOCAB_SIZE,
            targ_vocab_size=LABELS_SIZE,
            use_prenet=False,
            prenet_sizes=[512, 256],
            cbhg_gru_units=256,
            cbhg_filters=16,
            cbhg_projections=[128, 256],
            post_cbhg_layers_units=[256, 256],
            post_cbhg_use_batch_norm=True,
        )
        path = CBHG_PATH

    # Send model to `device` (GPU/CPU)
    model.to(device)

    # Instantiate the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    return path, model, optimizer

### Prepare Data


In [44]:
train_corpus = readFile(TRAIN_PATH)
val_corpus = readFile(VAL_PATH)

X_train = []
Y_train = []

X_val = []
Y_val = []

for sentence in train_corpus[:2]:
    # Clean each sentence in the corpus
    # Get the char list for each word in the sentence and its corresponding diacritics
    char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

    for i in range(len(char_list)):
        X_train.append(char_list[i])
        Y_train.append(diacritics_list[i])

X_train_padded = [
    torch.tensor([char_to_index[char] for char in sentence]) for sentence in X_train
]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [
    torch.tensor([diacritic_to_index[char] for char in sentence])
    for sentence in Y_train
]
y_train_padded = pad_sequence(y_train_padded, batch_first=True)


for sentence in val_corpus:
    # Clean each sentence in the corpus
    # Get the char list for each word in the sentence and its corresponding diacritics
    char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

    for i in range(len(char_list)):
        X_val.append(char_list[i])
        Y_val.append(diacritics_list[i])

X_val_padded = [
    torch.tensor([char_to_index[char] for char in sentence]) for sentence in X_val
]
X_val_padded = pad_sequence(X_val_padded, batch_first=True)

y_val_padded = [
    torch.tensor([diacritic_to_index[char] for char in sentence]) for sentence in Y_val
]
y_val_padded = pad_sequence(y_val_padded, batch_first=True)

val_inputs, test_inputs, val_labels, test_labels = train_test_split(
    X_val_padded, y_val_padded, test_size=0.5, random_state=42
)

### Execute


In [45]:
path, model, optimizer = initilize_model("RNN")
train_dataloader, val_dataloader = data_loader(
    X_train_padded, val_inputs, y_train_padded, val_labels
)
train(path, model, optimizer, train_dataloader, val_dataloader)

Start training...

 Epoch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
test_data = TensorDataset(test_inputs, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)
test_loss, test_acc = evaluate(model, test_dataloader)
print("Test Loss: {}".format(test_loss))
print("Test Accuracy: {}".format(test_acc))

Test Loss: 0.22198111241540083
Test Accuracy: 92.53917212516103
